# Context aware Neural network Model

## Convert PyTorch Models to TensorFlow 


In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.optim as optim
import tensorflow as tf
import torch.nn as nn
import numpy as np
import torch
import onnx
import time
import os
import sys
import cv2

In [1]:
from onnx_tf.backend import prepare

/home/ubuntu/mayub/Github/onnx-tensorflow/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


In [ ]:
sys.path.insert(1, '/home/ubuntu/mayub/Github/Context-Aware_Crowd_Counting-pytorch/')
import cannet as cann_model

In [ ]:
MODEL_PARAM_PATH = '/home/ubuntu/mayub/Github/Context-Aware_Crowd_Counting-pytorch/cvpr2019_CAN_SHHA_353.pth'

In [ ]:
model= cann_model.CANNet()
device=torch.device("cpu")
model.load_state_dict(torch.load(MODEL_PARAM_PATH, map_location=torch.device('cpu')))
model.to(device)
model.eval()
print(model)

### Preprocess the Image for CANNet model (context aware model)

In [ ]:
def read_and_convert_img(image_path):
    img=plt.imread(image_path)/255
    print(len(img.shape))
    if len(img.shape)==2:
        # expand grayscale image to three channel.
        img=img[:,:,np.newaxis]
        img=np.concatenate((img,img,img),2)
    print(img.shape[0])
    print(img.shape[1])
    ds_rows=int(img.shape[0]//8) # Downsampling to match model size
    ds_cols=int(img.shape[1]//8)
    print(ds_rows)
    print(ds_cols)
    img = cv2.resize(img,(ds_cols*8,ds_rows*8))
    print(img.shape)
    img=img.transpose((2,0,1)) # convert to order (channel,rows,cols)
    img_tensor=torch.tensor(img,dtype=torch.float)
    img_tensor=transforms.functional.normalize(img_tensor,mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    img_tensor=img_tensor.view(1,img_tensor.shape[0],img_tensor.shape[1],img_tensor.shape[2])
    print(img.shape)
    print(img_tensor.shape)
    #img_tensor = np.expand_dims(img_tensor,axis  = 0)
    return img_tensor

### Check Input image dimentions 

In [ ]:
image_input= read_and_convert_img('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_A_final/test_data/images/IMG_1.jpg')

### Run the Prediction on sample image to test model

In [ ]:
image_input=image_input.to(device)
et_dmap=model(image_input)
print(et_dmap.data.sum())

### Export the model in ONNX format

In [ ]:
# Export to ONNX format
torch.onnx.export(model, image_input, './model_simple.onnx', input_names=['image_input'], output_names=['image_output'],operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

### Prepare the model for TensorFlow export

In [4]:
# Load ONNX model and convert to TensorFlow format
model_onnx = onnx.load('./model_simple.onnx')

# tf_rep = prepare(model_onnx)

# Export model as .pb file
#tf_rep.export_graph('./model_simple.pb')

In [ ]:
tf_rep = prepare(model_onnx,device='CPU')

## FAILS !! Aten op is not implemented in ONNX 

In [3]:
prepare()

AttributeError: 'function' object has no attribute 'device'